In [ ]:
import os

os.environ["HF_HOME"] = "/root/autodl-tmp/HF_download"
os.environ["MODELSCOPE_CACHE"] = "/root/autodl-tmp/MODELSCOPE_download"
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [ ]:
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/MODELSCOPE_download/models/modelscope/Llama-2-7b-ms", trust_remote_code=True)

In [ ]:
from datasets import Dataset

datasets = Dataset.load_from_disk("/root/autodl-tmp/code/test-transformers/data/alpaca_data_zh")

In [ ]:
tokenizer.padding_side = "right"

In [ ]:
tokenizer.pad_token_id = 2

In [ ]:
def process_function(example):
  MAX_LENGTH = 1024
  instruction = tokenizer("\n".join([f"Human: " + example["instruction"].strip(), example["input"].strip()]).strip() + "\n\nAssistant: ", add_special_tokens=False)
  response = tokenizer(example["output"].strip(), add_special_tokens=False)
  input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
  attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
  labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
  if len(input_ids) > MAX_LENGTH:
    return {}
  return {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "labels": labels
  }

In [ ]:
len(datasets)

In [ ]:
tokenized_datasets = datasets.map(process_function, remove_columns=datasets.column_names)

In [ ]:
len(tokenized_datasets)

In [ ]:
tokenizer.decode(tokenized_datasets[0]["input_ids"])

In [ ]:
print(tokenized_datasets[0]["attention_mask"])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_datasets[0]["labels"])))

In [ ]:
import torch
import gc

model=None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
  "/root/autodl-tmp/MODELSCOPE_download/models/modelscope/Llama-2-7b-ms",
  dtype=torch.bfloat16,
  trust_remote_code=True,
  low_cpu_mem_usage=True,
  device_map="auto"
)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
  task_type=TaskType.CAUSAL_LM
)

In [ ]:
model = get_peft_model(model, config)

In [ ]:
for name, module in model.named_modules():
  if "lora_" in name:
    module.to(torch.bfloat16)

In [ ]:
# model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/root/autodl-tmp/code/test-transformers/test-kbit/16bits-training/llama-chatbot2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=1,
    report_to=["tensorboard"],
    # 关闭28.85G显存，开启16G显存
    # gradient_checkpointing=True,
    # adam_epsilon=1e-4, # 如果使用fp16的需要调大精度范围防止溢出
)

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
pipe("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ")